# Suguru 


> 规则：

- 每个大小为 $n$ 的区域必须包含从 1 到 n 的所有数字；
- 相邻的格子不能有同样的数字。这里的相邻格子指的是水平、竖直和对角线相邻的 8 个格子（如果有8个的话）。

> Rules: 


- Every region must contain the digits from 1 to the number of cells within that region.
- Neighbouring cells(When we talk about neighbouring cells in suguru, we mean any cell that touches another horizontally, vertically, or diagonally, 8 cells(if valid) in total) cannot contain the same digit.

<https://puzzlegenius.org/suguru-from-scratch/>




In [53]:
from ortools.sat.python import cp_model as cp

class SuguruSolver:
    
    def __init__(self, X, Y, grid, nums):
        self.X = X
        self.Y = Y 
        self.grid = grid
        self.nums = nums
        self.model = cp.CpModel()
        self.solver = cp.CpSolver()
        self.x = {}
        self.cells = {}
        # cells: {
        #    "cel name": [(xx, yy)] 
        # }
        for i in range(self.Y):
            for j in range(self.X):
                cur_cells = self.grid[i * self.Y + j]
                if self.grid[i * self.Y + j] not in self.cells:
                    self.cells[cur_cells] = [(i, j)]
                else:
                    self.cells[cur_cells].append((i, j))
        
        for i in range(self.Y):
            for j in range(self.X):
                if self.nums[i * self.Y  + j] == "0":
                    cur_cells = self.grid[i * self.Y + j]
                    self.x[i, j] = self.model.NewIntVar(1, len(self.cells[cur_cells]) , f'x[{i}, {j}]')
                else:
                    self.x[i, j] = int(self.nums[i * self.X + j])
                
    
    def addConstr(self):
        for _, pos in self.cells.items():
            cur_cell_digits = [self.x[i, j] for (i, j) in pos]
            self.model.AddAllDifferent(cur_cell_digits)
        
        
        def getNeighbours(x_, y_ ):
            res = []
            for ofsetx in range(-1, 2):
                for ofsety in range(-1, 2):
                    if (x_ + ofsetx >= 0 and x_ + ofsetx < self.Y ) and (y_ + ofsety >= 0 and y_ + ofsety < self.X) and (ofsetx != 0 or ofsety != 0):
                        res.append(self.x[x_ + ofsetx, y_ + ofsety])
            return res
            
        for i in range(self.Y):
            for j in range(self.X):
                neighbours = getNeighbours(i, j)
                for neighbour in neighbours: 
                    self.model.Add(self.x[i, j] != neighbour)

    def solve(self):
        self.addConstr()
        status = self.solver.Solve(self.model)
        if status == cp.OPTIMAL:
            for i in range(self.Y):
                for j in range(self.X):
                    print(self.solver.Value(self.x[i, j]), end=" ")
                print()
            print()

            print("NumConflicts:", self.solver.NumConflicts())
            print("NumBranches:", self.solver.NumBranches())
            print("WallTime:", self.solver.WallTime())


In [54]:


if __name__ == "__main__":
    grids = "AAABCCDDBBCCDDBEECDFBEEGFFHHEGFHHHGG"
    nums = "200402005050000000500003105200200301"
    suguruSolverTest = SuguruSolver(6, 6, grids, nums)
    suguruSolverTest.solve()
    print(suguruSolverTest.solver.ResponseStats())

2 3 1 4 1 2 
1 4 5 3 5 3 
2 3 2 4 2 4 
5 4 1 3 1 3 
1 3 5 2 5 2 
2 4 1 3 4 1 

NumConflicts: 0
NumBranches: 0
WallTime: 0.005803
CpSolverResponse summary:
status: OPTIMAL
objective: 0
best_bound: 0
integers: 0
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime: 0.005803
usertime: 0.005803
deterministic_time: 6.888e-05
gap_integral: 0
solution_fingerprint: 0x517a4888f62ffaeb



![](../assets/figures/Suguru.png)